In [141]:
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from pandas.core.common import flatten

import numpy as np
import re

import gensim
import string
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import pymorphy2




In [142]:
result = pd.read_csv('result.csv', sep=';', names=['param','guid','group','result'])
train = pd.read_csv('train.csv', sep=';', names=['param','guid','group'])
classes = pd.read_csv('classes.csv', sep=';', names=['param','group'])

In [143]:
train.head()

,param,guid,group
0,Количество фаз,a2d06669-a727-4e50-bcba-d7bb905d3372,NumberOfPoles
1,Ед. измерения,763aa9a4-e841-48f9-bcf5-2e64763184aa,Text
2,Единица измерения,9e75a863-eed0-4d52-a37d-49b818375f45,Text
3,Классификация нагрузок,799c12d4-7e40-4ac2-9d19-e2e9a61923ac,LoadClassification
4,n_фаз,0257a4b0-6a8d-42c3-bae6-334ab79a9818,Integer


In [144]:
result.head()

,param,guid,group,result
0,Количество фаз,a2d06669-a727-4e50-bcba-d7bb905d3372,NumberOfPoles,ADSK_Количество фаз
1,Ед. измерения,763aa9a4-e841-48f9-bcf5-2e64763184aa,Text,ADSK_Единица измерения
2,Единица измерения,9e75a863-eed0-4d52-a37d-49b818375f45,Text,ADSK_Единица измерения
3,Классификация нагрузок,799c12d4-7e40-4ac2-9d19-e2e9a61923ac,LoadClassification,ADSK_Классификация нагрузок
4,n_фаз,0257a4b0-6a8d-42c3-bae6-334ab79a9818,Integer,ADSK_Количество фаз


In [145]:
pd.merge(train, result, how='right', left_on='guid', right_on='guid').head()

,param_x,guid,group_x,param_y,group_y,result
0,Количество фаз,a2d06669-a727-4e50-bcba-d7bb905d3372,NumberOfPoles,Количество фаз,NumberOfPoles,ADSK_Количество фаз
1,Ед. измерения,763aa9a4-e841-48f9-bcf5-2e64763184aa,Text,Ед. измерения,Text,ADSK_Единица измерения
2,Единица измерения,9e75a863-eed0-4d52-a37d-49b818375f45,Text,Единица измерения,Text,ADSK_Единица измерения
3,Классификация нагрузок,799c12d4-7e40-4ac2-9d19-e2e9a61923ac,LoadClassification,Классификация нагрузок,LoadClassification,ADSK_Классификация нагрузок
4,n_фаз,0257a4b0-6a8d-42c3-bae6-334ab79a9818,Integer,n_фаз,Integer,ADSK_Количество фаз


In [146]:
classes.head()

,param,group
0,ADSK_Единица измерения,TEXT
1,ADSK_Завод-изготовитель,TEXT
2,ADSK_Зона,TEXT
3,ADSK_Код изделия,TEXT
4,ADSK_Марка,TEXT


## НУЖНО РАСПЛИТИТЬ ПО БУКВАМ CLASSES и TRAIN

In [147]:
re.findall('[A-Z][^A-Z]*', 'TheLongAndWindingRoad')
re.findall('[A-Z][^A-Z]*', 'TTT')
re.findall('[А-ЯЁ][^А-ЯЁ]*', 'УголПоворота')

re.findall('[А-ЯЁ][^А-ЯЁ]*', 'Размер УголПоворота')


['Размер ', 'Угол', 'Поворота']

In [148]:
classes['source'] = classes['param']
classes['param'] = classes['param'].str.split('_').apply(lambda x: x[1:])


In [149]:
classes.head()

,param,group,source
0,[Единица измерения],TEXT,ADSK_Единица измерения
1,[Завод-изготовитель],TEXT,ADSK_Завод-изготовитель
2,[Зона],TEXT,ADSK_Зона
3,[Код изделия],TEXT,ADSK_Код изделия
4,[Марка],TEXT,ADSK_Марка


### Функция обработки

In [150]:
def splitting(Ser):
    Ser = Ser.apply(lambda x: [re.findall('[А-ЯЁ][^А-ЯЁ]*', y) for y in x])
    Ser = Ser.apply(lambda x: [re.split(';|,|\*|\n|\s|_|-', y) for y in list(flatten(x))])
    Ser = Ser.apply(lambda x: list(flatten(x)))
    Ser = Ser.apply(lambda x: [y.lower() for y in x if (len(y) != 1) and (y.isdigit() is not True)])
    Ser = Ser.str.join(' ')
#     Ser = Ser.apply(lambda x: x[0] if type(x) == list else x)
    return Ser

### Обрабатываем classes

In [151]:
## тест
classes['param'].apply(lambda x: [re.findall('[А-ЯЁ][^А-ЯЁ]*', y) for y in x])
# classes['param'].apply(lambda x: re.split(';|,|\*|\n|\s|_|-', x))

0               [[Единица измерения]]
1              [[Завод-изготовитель]]
2                            [[Зона]]
3                     [[Код изделия]]
4                           [[Марка]]
                    ...              
241             [[Размер], [Глубина]]
242    [[Размер], [Диаметр, Изделия]]
243      [[Размер], [Толщина основы]]
244      [[Размер], [Угол, Поворота]]
245              [[Размер], [Радиус]]
Name: param, Length: 246, dtype: object

In [152]:
classes['vector'] = splitting(classes['param'])
print(len(classes))
# classes['vector'].apply(type).value_counts()


246


,param,group,source,vector
0,[Единица измерения],TEXT,ADSK_Единица измерения,единица измерения
1,[Завод-изготовитель],TEXT,ADSK_Завод-изготовитель,завод изготовитель
2,[Зона],TEXT,ADSK_Зона,зона
3,[Код изделия],TEXT,ADSK_Код изделия,код изделия
4,[Марка],TEXT,ADSK_Марка,марка
...,...,...,...,...
241,"[Размер, Глубина]",LENGTH,ADSK_Размер_Глубина,размер глубина
242,"[Размер, ДиаметрИзделия]",LENGTH,ADSK_Размер_ДиаметрИзделия,размер диаметр изделия
243,"[Размер, Толщина основы]",LENGTH,ADSK_Размер_Толщина основы,размер толщина основы
244,"[Размер, УголПоворота]",ANGLE,ADSK_Размер_УголПоворота,размер угол поворота


In [153]:
pd.Series(list(flatten(classes['vector'].to_list()))).value_counts()

                                    20
штамп  строка должность              6
штамп  боковой  строка фамилия       6
штамп  строка фамилия                6
штамп  боковой  строка должность     6
                                    ..
свободный напор жидкости             1
выступ метиза                        1
индекс квартиры                      1
расход воздуха вытяжной              1
толщина стены                        1
Length: 202, dtype: int64

In [159]:
# classes['vector'] = classes['vector'].str.join(' ')
classes['param'].iloc[0]

list

In [155]:
classes =classes.reset_index()

### Обрабатываем train

In [156]:
train['param'] = train['param']
train['vector'] = splitting(train['param'])
train

,param,guid,group,vector
0,Количество фаз,a2d06669-a727-4e50-bcba-d7bb905d3372,NumberOfPoles,
1,Ед. измерения,763aa9a4-e841-48f9-bcf5-2e64763184aa,Text,
2,Единица измерения,9e75a863-eed0-4d52-a37d-49b818375f45,Text,
3,Классификация нагрузок,799c12d4-7e40-4ac2-9d19-e2e9a61923ac,LoadClassification,
4,n_фаз,0257a4b0-6a8d-42c3-bae6-334ab79a9818,Integer,
...,...,...,...,...
1001,Мощность,6f6a6a94-13ad-4946-ba66-fac6566f0033,HVACPower,
1002,Толщина 1,736dabe1-ea07-4213-ba84-afe395de2820,PipeSize,
1003,"Тип, марка, обозначение документа, опросного л...",90214fe6-6311-44cc-be60-55b6a63c6418,Text,
1004,"Масса единицы, кг",9e2f55f6-15fe-4892-af03-4a0dc62bd468,Text,


### Создаем модель и обучаем на Classes (ADSK параметрах)

In [160]:
text_clf = Pipeline([
                     ('tfidf', TfidfVectorizer()),
                     ('clf', RandomForestClassifier())
                     ])

In [161]:
text_clf.fit(classes['vector'], classes['index'])

C:\Users\r.enikeev\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='...
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                               

## ПРЕДИКТ

In [162]:
 
# res = text_clf.predict(['текст номер три'])
# print(res)  # [1]


res = text_clf.predict(train['param'])
train['predict'] = res
train

,param,guid,group,vector,predict
0,Количество фаз,a2d06669-a727-4e50-bcba-d7bb905d3372,NumberOfPoles,,54
1,Ед. измерения,763aa9a4-e841-48f9-bcf5-2e64763184aa,Text,,0
2,Единица измерения,9e75a863-eed0-4d52-a37d-49b818375f45,Text,,0
3,Классификация нагрузок,799c12d4-7e40-4ac2-9d19-e2e9a61923ac,LoadClassification,,53
4,n_фаз,0257a4b0-6a8d-42c3-bae6-334ab79a9818,Integer,,127
...,...,...,...,...,...
1001,Мощность,6f6a6a94-13ad-4946-ba66-fac6566f0033,HVACPower,,73
1002,Толщина 1,736dabe1-ea07-4213-ba84-afe395de2820,PipeSize,,243
1003,"Тип, марка, обозначение документа, опросного л...",90214fe6-6311-44cc-be60-55b6a63c6418,Text,,11
1004,"Масса единицы, кг",9e2f55f6-15fe-4892-af03-4a0dc62bd468,Text,,5


In [174]:
train[train['predict'] == 127]          #classes с индексом 0 тоже Завод-изготовитель

,param,guid,group,vector,predict
4,n_фаз,0257a4b0-6a8d-42c3-bae6-334ab79a9818,Integer,,127
11,ГОСТ (ТУ),55f83ad4-d5c2-45d7-bae4-621e45beb584,Text,,127
21,GP_Наименование,d4ae8837-57f3-462d-993e-2b9476a5db01,Text,,127
22,GP_Обозначение,6e21897e-fedd-4e81-8e15-28428d291702,Text,,127
23,GP_Примечание,be30076a-bfd8-4bba-9018-d38b6f3dc659,Text,,127
...,...,...,...,...,...
975,Масштаб,03b75860-d6dd-410f-8a50-df76389bf77f,Number,,127
976,Специальность,d827ae68-7539-479b-a576-264c341a58d0,Text,,127
982,Сортировка в спецификации по принадлежности к ...,b00c342e-21a4-424b-8c97-519f05abc8da,Integer,,127
985,Номинальное Давление,3ec34236-c6cd-4b3e-8679-fff71527374a,PipingPressure,,127


In [164]:
classes

,index,param,group,source,vector
0,0,[Единица измерения],TEXT,ADSK_Единица измерения,единица измерения
1,1,[Завод-изготовитель],TEXT,ADSK_Завод-изготовитель,завод изготовитель
2,2,[Зона],TEXT,ADSK_Зона,зона
3,3,[Код изделия],TEXT,ADSK_Код изделия,код изделия
4,4,[Марка],TEXT,ADSK_Марка,марка
...,...,...,...,...,...
241,241,"[Размер, Глубина]",LENGTH,ADSK_Размер_Глубина,размер глубина
242,242,"[Размер, ДиаметрИзделия]",LENGTH,ADSK_Размер_ДиаметрИзделия,размер диаметр изделия
243,243,"[Размер, Толщина основы]",LENGTH,ADSK_Размер_Толщина основы,размер толщина основы
244,244,"[Размер, УголПоворота]",ANGLE,ADSK_Размер_УголПоворота,размер угол поворота


In [72]:
# train['param'] = train['param'].str.join(' ')


In [166]:
pred = pd.merge(train, classes, how='left', left_on='predict', right_on='index')
pred
# pred['param_y'] = 'ADSK_' + pred['param_y']
# pred[['param_x', 'param_y']]
# pred[pred.guid == 'a2d06669-a727-4e50-bcba-d7bb905d3372']
# len(pred)

,param_x,guid,group_x,vector_x,predict,index,param_y,group_y,source,vector_y
0,Количество фаз,a2d06669-a727-4e50-bcba-d7bb905d3372,NumberOfPoles,,54,54,[Количество фаз],NOOFPOLES,ADSK_Количество фаз,количество фаз
1,Ед. измерения,763aa9a4-e841-48f9-bcf5-2e64763184aa,Text,,0,0,[Единица измерения],TEXT,ADSK_Единица измерения,единица измерения
2,Единица измерения,9e75a863-eed0-4d52-a37d-49b818375f45,Text,,0,0,[Единица измерения],TEXT,ADSK_Единица измерения,единица измерения
3,Классификация нагрузок,799c12d4-7e40-4ac2-9d19-e2e9a61923ac,LoadClassification,,53,53,[Классификация нагрузок],LOADCLASSIFICATION,ADSK_Классификация нагрузок,классификация нагрузок
4,n_фаз,0257a4b0-6a8d-42c3-bae6-334ab79a9818,Integer,,127,127,[C],REINFORCEMENT_LENGTH,ADSK_C,
...,...,...,...,...,...,...,...,...,...,...
1001,Мощность,6f6a6a94-13ad-4946-ba66-fac6566f0033,HVACPower,,73,73,[Тепловая мощность],HVAC_POWER,ADSK_Тепловая мощность,тепловая мощность
1002,Толщина 1,736dabe1-ea07-4213-ba84-afe395de2820,PipeSize,,243,243,"[Размер, Толщина основы]",LENGTH,ADSK_Размер_Толщина основы,размер толщина основы
1003,"Тип, марка, обозначение документа, опросного л...",90214fe6-6311-44cc-be60-55b6a63c6418,Text,,11,11,[Обозначение],TEXT,ADSK_Обозначение,обозначение
1004,"Масса единицы, кг",9e2f55f6-15fe-4892-af03-4a0dc62bd468,Text,,5,5,[Масса],NUMBER,ADSK_Масса,масса


In [74]:
pred.head()

,param_x,guid,group_x,predict,index,param_y,group_y
0,Количество фаз,a2d06669-a727-4e50-bcba-d7bb905d3372,NumberOfPoles,54,54,ADSK_Количество фаз,NOOFPOLES
1,Ед. измерения,763aa9a4-e841-48f9-bcf5-2e64763184aa,Text,0,0,ADSK_Единица измерения,TEXT
2,Единица измерения,9e75a863-eed0-4d52-a37d-49b818375f45,Text,0,0,ADSK_Единица измерения,TEXT
3,Классификация нагрузок,799c12d4-7e40-4ac2-9d19-e2e9a61923ac,LoadClassification,53,53,ADSK_Классификация нагрузок,LOADCLASSIFICATION
4,n_фаз,0257a4b0-6a8d-42c3-bae6-334ab79a9818,Integer,131,131,ADSK_G,REINFORCEMENT_LENGTH


In [173]:
train[train['guid'] == '5bceeb6e-a0d2-43f2-94a6-7081d89be3b3']

,param,guid,group,vector,predict
30,PF_Угол,5bceeb6e-a0d2-43f2-94a6-7081d89be3b3,Angle,,127


In [76]:
result

,param,guid,group,result
0,Количество фаз,a2d06669-a727-4e50-bcba-d7bb905d3372,NumberOfPoles,ADSK_Количество фаз
1,Ед. измерения,763aa9a4-e841-48f9-bcf5-2e64763184aa,Text,ADSK_Единица измерения
2,Единица измерения,9e75a863-eed0-4d52-a37d-49b818375f45,Text,ADSK_Единица измерения
3,Классификация нагрузок,799c12d4-7e40-4ac2-9d19-e2e9a61923ac,LoadClassification,ADSK_Классификация нагрузок
4,n_фаз,0257a4b0-6a8d-42c3-bae6-334ab79a9818,Integer,ADSK_Количество фаз
...,...,...,...,...
120,Ширина,5bee8e1a-2928-4464-ad47-2f05d24d8505,Length,ADSK_Размер_Ширина
121,Номер помещения,d0efb795-ce89-4a3c-a7f7-191acf6e5504,Text,ADSK_Номер помещения квартиры
122,Мощность,6f6a6a94-13ad-4946-ba66-fac6566f0033,HVACPower,ADSK_Тепловая мощность
123,"Масса единицы, кг",9e2f55f6-15fe-4892-af03-4a0dc62bd468,Text,ADSK_Масса


In [168]:
pred = pred[['param_x', 'guid', 'param_y']]
pred.columns = ['source', 'guid', 'predict_p']
pred[pred.guid == 'bb4022c6-e79b-4540-959e-f9f851ebb1f8']
# pred

,source,guid,predict_p
1005,"Код оборудования, изделия, материала",bb4022c6-e79b-4540-959e-f9f851ebb1f8,[Код перегородки]


In [78]:
result.head()

,param,guid,group,result
0,Количество фаз,a2d06669-a727-4e50-bcba-d7bb905d3372,NumberOfPoles,ADSK_Количество фаз
1,Ед. измерения,763aa9a4-e841-48f9-bcf5-2e64763184aa,Text,ADSK_Единица измерения
2,Единица измерения,9e75a863-eed0-4d52-a37d-49b818375f45,Text,ADSK_Единица измерения
3,Классификация нагрузок,799c12d4-7e40-4ac2-9d19-e2e9a61923ac,LoadClassification,ADSK_Классификация нагрузок
4,n_фаз,0257a4b0-6a8d-42c3-bae6-334ab79a9818,Integer,ADSK_Количество фаз


In [169]:
analyz = pd.merge(result, pred, how='left', left_on='guid', right_on='guid')
analyz.head()

,param,guid,group,result,source,predict_p
0,Количество фаз,a2d06669-a727-4e50-bcba-d7bb905d3372,NumberOfPoles,ADSK_Количество фаз,Количество фаз,[Количество фаз]
1,Ед. измерения,763aa9a4-e841-48f9-bcf5-2e64763184aa,Text,ADSK_Единица измерения,Ед. измерения,[Единица измерения]
2,Единица измерения,9e75a863-eed0-4d52-a37d-49b818375f45,Text,ADSK_Единица измерения,Единица измерения,[Единица измерения]
3,Классификация нагрузок,799c12d4-7e40-4ac2-9d19-e2e9a61923ac,LoadClassification,ADSK_Классификация нагрузок,Классификация нагрузок,[Классификация нагрузок]
4,n_фаз,0257a4b0-6a8d-42c3-bae6-334ab79a9818,Integer,ADSK_Количество фаз,n_фаз,[C]


In [170]:
analyz[['param', 'guid', 'result', 'predict_p']].to_excel('mytry.xlsx')

In [171]:
sum(analyz['result'] == analyz['predict_p'])/len(analyz)

0.0

In [22]:
result
pred
len(pd.merge(result, pred, how='left', left_on='guid', right_on='guid'))
# len(ready), len(result)
pd.merge(result, pred, how='inner', left_on='guid', right_on='guid')

,param,guid,group,result,param_x,group_x,predict,index,param_y,group_y


## КОНЕЦ


In [21]:
pred[['param_x', 'param_y', 'guid']].to_excel('mytry.xlsx', encoding='utf-8')

In [ ]:
train.iloc[2, 0]
len(train['param'])

In [ ]:
train.iloc[0, 0]

In [ ]:

## возвращает не всё, на BIM test вернет BIM
def split_only_upper(str_in):
    word = ""
    for i in range(0, len(str_in)):
        if i+1 != len(str_in) and str_in[i].isupper() and str_in[i + 1].isupper() and word:
#             yield word
#             word = str_in[i]
#             continue
            word += str_in[i]
    yield word
def split_by_upper(string):
    word = ""
    for letter in string:
        if letter.isupper() and word:
            yield word
            word = letter
            continue
        word += letter
    yield word

In [ ]:
train['param'] = train['param'].str.split(r';|,|\*|\n|\s|_')

train

In [ ]:
result = []
clear = list(set(re.split(';|,|\*|\n|\s|_', train['param'].to_string())))

# [split_by_upper(x) for x in clear]
for word in clear:
#     pass
    
#     print(*split_by_upper(word))
    if not word.isdigit():

        result.append(word)
result
# clear[0]
# [i.lower() for i in clear]
# clear = map(clear, lambda x: x.lower())


In [ ]:

test = 'НоминальныйДиаметр'
test2 = 'NominalDiameter'
test3 = 'BIM'
test4 = '94'
print(test4.isdigit())
print(list(split_only_upper(test3)))
# re.split('\*[А-Я]', test)
# re.findall(r'\d([А-Я][А-Яа-я]+)([А-Я][А-Яа-я]+)', test)

In [ ]:
re.findall(r'[A-Z]?[^A-Z]*', test2)

In [ ]:
re.findall(r'[А-Я]?[^А-Я]*', test)

In [ ]:
string = "QuackТестЗалупаЖопаBeepBoop"                                                                                                                                                                   

def split_by_upper(string): 
    word = "" 
    for letter in string: 
        if letter.isupper() and word: 
            yield word 
            word = letter 
            continue 
        word += letter 
    yield word 
                                                                                                                                                                                       

[word for word in split_by_upper(string)]                                                                                                                                                                
#['Quack', 'Тест', 'Залупа', 'Жопа', 'Beep', 'Boop']